In [ ]:
from pathlib import Path

import iris
import matplotlib.pyplot as plt
import numpy as np
from utils import Alpha, Antecedent, Memory

In [ ]:
lai = iris.load_cube(
    str(
        Path(
            "~/JULES_data/lai_Lmon_UKESM1-0-LL_esm-hist_r1i1p1f2_gn_195001-201412.nc"
        ).expanduser()
    )
)
lai

In [ ]:
lai.coord("time").cell(0), lai.coord("time").cell(1)

In [ ]:
np.where(~lai.data.mask[0])

In [ ]:
fig_kwargs = dict(figsize=(9, 6), dpi=150)

In [ ]:
N = 24
data = lai.data[:N, 27, 154]
months = np.arange(1, N + 1)

for alpha, antec, period in ([0.1, 2, 2], [0.3, 1, 1]):
    alpha = Alpha(alpha, data)
    antecedent = Antecedent(antec, data)
    memory = Memory(data, period)
    plt.figure(**fig_kwargs)
    plt.plot(months, data, label="monthly data")
    plt.plot(months, alpha.weighted, label=fr"$\alpha = {{{alpha.alpha:0.3f}}}$")
    plt.plot(
        months, antecedent.antec, label=f"antec n={antecedent.antec_samples:d} month"
    )
    plt.plot(
        months, memory.data, label=f"disc n={memory.period:d} month", linestyle="--"
    )
    plt.xlabel("months")
    plt.ylabel("LAI")
    plt.grid()
    plt.title("Monthly Timestep")
    _ = plt.legend()

In [ ]:
N = 24
data = lai.data[:N, 27, 154]
months = np.arange(1, N + 1)

# Days, approximately.
days = np.linspace(1, N, N * 30)

daily_data = np.interp(days, months, data)

for alpha, antec, period in ([0.004, 60, 60], [0.03, 30, 30]):
    alpha = Alpha(alpha, daily_data)
    antecedent = Antecedent(antec, daily_data)
    memory = Memory(daily_data, period)
    plt.figure(**fig_kwargs)
    plt.plot(days, daily_data, label="daily data")
    plt.plot(days, alpha.weighted, label=fr"$\alpha = {{{alpha.alpha:0.3f}}}$")
    plt.plot(
        days,
        antecedent.antec,
        label=f"antec n={antecedent.antec_samples:d} "
        f"days\n({antecedent.antec_samples / 30:.1f} months)",
    )
    plt.plot(
        days,
        memory.data,
        label=f"disc n={memory.period:d} days\n"
        f"({memory.period / 30 : 0.1f} months)",
        linestyle="--",
    )
    plt.xlabel("months")
    plt.ylabel("LAI")
    plt.grid()
    plt.title("Daily Timestep")
    _ = plt.legend()